In [5]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import geopandas as gpd

import logging
pd.set_option('display.max_columns', None)
logging.getLogger().setLevel(logging.INFO) # DEBUG, INFO or WARN
DROOT = '../1-data/'

In [3]:
# Loading this file might take a moment.
urbancenters = gpd.read_file(
    os.path.join(DROOT, '2-external', 'GHS_STAT_UCDB2015MT_GLOBE_R2019A', 'GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg'))
urbancenters.head(2)

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,CTR_MN_ISO,XBRDR,XCTR_NBR,XC_NM_LST,XC_ISO_LST,GRGN_L1,GRGN_L2,UC_NM_MN,UC_NM_LST,UC_NM_SRC,H75_NBR,H90_NBR,H00_NBR,H75_AREA,H90_AREA,H00_AREA,E_BM_NM_LST,E_SL_LST,EL_AV_ALS,E_KG_NM_LST,E_RB_NM_LST,E_WR_P_90,E_WR_P_00,E_WR_P_14,E_WR_T_90,E_WR_T_00,E_WR_T_14,B75,B90,B00,B15,P75,P90,P00,P15,BUCAP75,BUCAP90,BUCAP00,BUCAP15,NTL_AV,GDP90_SM,GDP00_SM,GDP15_SM,INCM_CMI,DEV_CMI,TT2CC,E_GR_AV90,E_GR_AV00,E_GR_AV14,E_GR_AH90,E_GR_AM90,E_GR_AL90,E_GR_AT90,E_GR_AH00,E_GR_AM00,E_GR_AL00,E_GR_AT00,E_GR_AH14,E_GR_AM14,E_GR_AL14,E_GR_AT14,E_EC2E_E75,E_EC2E_E90,E_EC2E_E00,E_EC2E_E15,E_EC2E_R75,E_EC2E_R90,E_EC2E_R00,E_EC2E_R15,E_EC2E_I75,E_EC2E_I90,E_EC2E_I00,E_EC2E_I15,E_EC2E_T75,E_EC2E_T90,E_EC2E_T00,E_EC2E_T15,E_EC2E_A75,E_EC2E_A90,E_EC2E_A00,E_EC2E_A15,E_EC2O_E75,E_EC2O_E90,E_EC2O_E00,E_EC2O_E15,E_EC2O_R75,E_EC2O_R90,E_EC2O_R00,E_EC2O_R15,E_EC2O_I75,E_EC2O_I90,E_EC2O_I00,E_EC2O_I15,E_EC2O_T75,E_EC2O_T90,E_EC2O_T00,E_EC2O_T15,E_EC2O_A75,E_EC2O_A90,E_EC2O_A00,E_EC2O_A15,E_EPM2_E75,E_EPM2_E90,E_EPM2_E00,E_EPM2_E15,E_EPM2_R75,E_EPM2_R90,E_EPM2_R00,E_EPM2_R15,E_EPM2_I75,E_EPM2_I90,E_EPM2_I00,E_EPM2_I15,E_EPM2_T75,E_EPM2_T90,E_EPM2_T00,E_EPM2_T15,E_EPM2_A75,E_EPM2_A90,E_EPM2_A00,E_EPM2_A15,E_CPM2_T00,E_CPM2_T05,E_CPM2_T10,E_CPM2_T14,EX_FD_AREA,EX_FD_B75,EX_FD_B90,EX_FD_B00,EX_FD_B15,EX_FD_P75,EX_FD_P90,EX_FD_P00,EX_FD_P15,EX_SS_AREA,EX_SS_B75,EX_SS_B90,EX_SS_B00,EX_SS_B15,EX_SS_P75,EX_SS_P90,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX,geometry
0,1.0,1.0,185.0,21.247683,-158.043016,21.422193,-157.730529,21.340678,-157.893497,United States,USA,0.0,1.0,United States,USA,Northern America,Northern America,Honolulu,Honolulu; Waipahu; Pearl City; Aiea,GISCO OSM,1.0,1.0,1.0,115.0,152.0,174.0,Tropical and Subtropical Dry Broadleaf Forests,Vertisols,52.296435,NAN,NAN,955.500031,620.975021,741.625015,23.456032,23.153346,23.526622,78.845200,78.845200,80.214867,80.647377,277211.196645,378482.072123,458967.881664,512853.666675,284.422854,208.319509,174.772288,157.252219,24.768574,1.504144e+10,1.584126e+10,2.192668e+10,HIC,MDR,15412.057919,0.231515,0.349464,0.369290,3.310453,129.41213,51.089083,179.678081,56.331924,113.192717,14.287026,183.811667,70.317059,98.455611,15.038997,183.811667,2.867485e+06,1.265081e+06,1.012234e+06,495299.593436,456114.038821,403581.138607,462973.483368,396627.287294,598840.500466,601347.375094,541277.833862,456837.498271,208651.075028,276910.085019,341852.064389,345201.327942,0.013626,0.029494,0.042942,0.056613,NaN,156371.556763,42922.159473,11381.377166,NaN,13811.961727,11022.504909,8225.571851,NaN,29257.399703,67747.537985,51154.996318,NaN,NaN,1029.574596,10692.735959,2.418542,3.153900,2.293634,3.078804,300.287590,134.342385,98.071016,41.850553,88.248068,112.991622,68.033587,40.144556,118.767334,208.712524,224.997453,174.221274,29.250431,43.893797,29.364700,23.536766,0.042916,0.043009,0.041517,0.047534,7.237346,6.158930,5.563796,5.765125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,161.0,68.280144,68.280144,69.576241,69.95092,240002.900655,327680.831522,397443.031445,444041.529529,NaN,NaN,missing,NaN,0.074385,0.226415,56.41,"MULTIPOLYGON (((-158.01244 21.42219, -157.9915..."
1,2.0,2.0,42.0,-17.641184,-149.628088,-17.517631,-149.508018,-17.534103,-149.568053,French Polynesia,PYF,0.0,1.0,French Polynesia,PYF,Oceania,Polynesia,Papeete,Papeete,GISCO OSM,0.0,1.0,1.0,0.0,29.0,29.0,Tropical and Subtropical Moist Broadleaf Forests,NAN,33.690318,NAN,NAN,1623.150055,1376.275009,1536.075012,21.907134,22.041168,22.021633,14.051702,14.051702,14.051702,14.493433,47644.964024,71862.231544,83726.092071,91521.124603,294.925221,195.536671,167.829421,158.361614,9.028501,0.000000e+00,0.000000e+00,0.000000e+00,HIC,LDC,NaN,NaN,0.445268,0.436477,0.000000,0.00000,0.000000,0.000000,15.656925,19.369919,6.614911,41.478138,16.868909,18.625840,6.147005,41.641755,3.240516e+03,2.106038e+04,1.506982e+04,4916.763772,4983.827260,166

In [36]:
cities = pd.read_csv(os.path.join(DROOT, '1-research', 'cities.latest.csv'), index_col=False)
cities.sort_values('priority').head(2)

,city_name,country_id,city_id,priority,n_cells,n_req,n_req_ok,frac_req_ok,city_centr_lat,city_centr_lon,region
0,Amsterdam,NLD,2167,0,345.0,5520.0,5520.0,1.0,52.355712,4.850578,Europe
10,s-Hertogenbosch,NLD,2194,0,45.0,720.0,720.0,1.0,51.708847,5.315975,Europe


In [22]:
# Initiate clean output list to add to excel.
cities_out = []

In [20]:
# Actually search cities here: 
city_name = "Charleroi"
search = urbancenters[urbancenters.UC_NM_LST.str.contains(city_name)]
search[['ID_HDC_G0', 'UC_NM_MN', 'CTR_MN_ISO', 'UC_NM_LST', 'AREA']]

,ID_HDC_G0,UC_NM_MN,CTR_MN_ISO,UC_NM_LST,AREA
2141,2142.0,Charleroi,BEL,Charleroi,124.0


In [23]:
# Pass the ID_HDC_G0 in here below:
ID_HDC_G0 = 2142.0
record = urbancenters[urbancenters.ID_HDC_G0 == ID_HDC_G0]
cities_out.append(record[['ID_HDC_G0', 'CTR_MN_ISO', 'UC_NM_MN', 'GCPNT_LAT','GCPNT_LON', 'GRGN_L1']].iloc[0].to_list())
record

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,CTR_MN_ISO,XBRDR,XCTR_NBR,XC_NM_LST,XC_ISO_LST,GRGN_L1,GRGN_L2,UC_NM_MN,UC_NM_LST,UC_NM_SRC,H75_NBR,H90_NBR,H00_NBR,H75_AREA,H90_AREA,H00_AREA,E_BM_NM_LST,E_SL_LST,EL_AV_ALS,E_KG_NM_LST,E_RB_NM_LST,E_WR_P_90,E_WR_P_00,E_WR_P_14,E_WR_T_90,E_WR_T_00,E_WR_T_14,B75,B90,B00,B15,P75,P90,P00,P15,BUCAP75,BUCAP90,BUCAP00,BUCAP15,NTL_AV,GDP90_SM,GDP00_SM,GDP15_SM,INCM_CMI,DEV_CMI,TT2CC,E_GR_AV90,E_GR_AV00,E_GR_AV14,E_GR_AH90,E_GR_AM90,E_GR_AL90,E_GR_AT90,E_GR_AH00,E_GR_AM00,E_GR_AL00,E_GR_AT00,E_GR_AH14,E_GR_AM14,E_GR_AL14,E_GR_AT14,E_EC2E_E75,E_EC2E_E90,E_EC2E_E00,E_EC2E_E15,E_EC2E_R75,E_EC2E_R90,E_EC2E_R00,E_EC2E_R15,E_EC2E_I75,E_EC2E_I90,E_EC2E_I00,E_EC2E_I15,E_EC2E_T75,E_EC2E_T90,E_EC2E_T00,E_EC2E_T15,E_EC2E_A75,E_EC2E_A90,E_EC2E_A00,E_EC2E_A15,E_EC2O_E75,E_EC2O_E90,E_EC2O_E00,E_EC2O_E15,E_EC2O_R75,E_EC2O_R90,E_EC2O_R00,E_EC2O_R15,E_EC2O_I75,E_EC2O_I90,E_EC2O_I00,E_EC2O_I15,E_EC2O_T75,E_EC2O_T90,E_EC2O_T00,E_EC2O_T15,E_EC2O_A75,E_EC2O_A90,E_EC2O_A00,E_EC2O_A15,E_EPM2_E75,E_EPM2_E90,E_EPM2_E00,E_EPM2_E15,E_EPM2_R75,E_EPM2_R90,E_EPM2_R00,E_EPM2_R15,E_EPM2_I75,E_EPM2_I90,E_EPM2_I00,E_EPM2_I15,E_EPM2_T75,E_EPM2_T90,E_EPM2_T00,E_EPM2_T15,E_EPM2_A75,E_EPM2_A90,E_EPM2_A00,E_EPM2_A15,E_CPM2_T00,E_CPM2_T05,E_CPM2_T10,E_CPM2_T14,EX_FD_AREA,EX_FD_B75,EX_FD_B90,EX_FD_B00,EX_FD_B15,EX_FD_P75,EX_FD_P90,EX_FD_P00,EX_FD_P15,EX_SS_AREA,EX_SS_B75,EX_SS_B90,EX_SS_B00,EX_SS_B15,EX_SS_P75,EX_SS_P90,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX,geometry
2141,2142.0,1.0,124.0,50.37801,4.324995,50.483401,4.561902,50.427604,4.440296,Belgium,BEL,0.0,1.0,Belgium,BEL,Europe,Western Europe,Charleroi,Charleroi,GISCO OSM,1.0,1.0,1.0,109.0,110.0,113.0,Temperate Broadleaf and Mixed Forests,Luvisols,138.942336,"Mild temperate, fully humid, and Warm summer",Rhine,882.325027,1031.050018,910.000015,10.764364,11.23606,11.157451,76.243805,79.390549,82.504753,84.035049,275881.510849,258483.99408,250645.475586,253691.458588,276.364316,307.139129,329.16913,331.249029,22.458066,2.579022e+09,3.043284e+09,3.408615e+09,HIC,MDR,19.201487,0.370928,0.443241,0.491237,30.682228,90.844812,2.552402,124.079441,63.898503,58.266637,1.914301,124.079441,85.071939,39.007502,0.0,124.079441,902173.958748,1.724623e+06,1.240471e+06,530129.409486,973731.271374,692659.769619,725663.654916,583323.601654,7.828588e+06,6.473675e+06,5.545647e+06,2.185090e+06,360869.636428,535005.195975,670151.68727,780186.196668,33.765788,13.172828,31.692245,45.943026,NaN,50598.032748,64352.591524,471777.827146,NaN,7452.167269,5920.009525,19100.07184,NaN,17995.47145,17673.137162,52317.85044,NaN,NaN,NaN,16767.13143,579.677661,807.158837,769.564079,846.810201,232.625477,261.257487,43.23044,118.670645,482.310531,149.392194,62.734454,50.437092,2740.403212,3557.922735,3675.209207,911.829634,68.931646,149.691395,209.183819,78.587937,4.445168,5.356238,5.362496,5.295084,16.89768,15.78012,14.94486,14.18745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.176517,5.0,available,8.95412,-3.0379,0.515178,68.61,"MULTIPOLYGON (((4.35312 50.48340, 4.36635 50.4..."


In [42]:
cities_new = pd.DataFrame(cities_out, columns=['city_id', 'country_id', 'city_name', 'city_centr_lat', 'city_centr_lon', 'region'])
cities_new['priority'] = 11 # 11 for normal, 0 for original collection, -1 for priority.
cities_new[['n_cells','n_req','n_req_ok','frac_req_ok']] = None
cities_new

,city_id,country_id,city_name,city_centr_lat,city_centr_lon,region,priority,n_cells,n_req,n_req_ok,frac_req_ok
0,2142.0,BEL,Charleroi,50.427604,4.440296,Europe,11,None,None,None,None


In [46]:
# Write out to file
cities_v2 = pd.concat([cities, cities_new])
cities_v2 = cities_v2.groupby('city_id').apply(lambda x: x.sort_values('priority', ascending=True).iloc[0])
cities_v2 = cities_v2.sort_values(['priority', 'country_id', 'city_name'])
cities_v2.to_csv('../1-data/1-research/cities.latest.csv')
cities_v2.tail(5)

,city_name,country_id,city_id,priority,n_cells,n_req,n_req_ok,frac_req_ok,city_centr_lat,city_centr_lon,region
city_id,,,,,,,,,,,
855.0,Washington D.C.,USA,855.0,11,1550.0,24800.0,0.0,0.0,38.922840,-77.141963,Northern America
513.0,Waterloo,USA,513.0,11,64.0,1024.0,0.0,0.0,42.497143,-92.373405,Northern America
11800.0,Ho Chi Minh City,VNM,11800.0,11,1467.0,23472.0,0.0,0.0,10.851848,106.712677,Asia
3268.0,Cape Town,ZAF,3268.0,11,697.0,11152.0,0.0,0.0,-33.947648,18.578423,Africa
3673.0,Johannesburg,ZAF,3673.0,11,1638.0,26208.0,0.0,0.0,-26.179771,28.055536,Africa


In [ ]:
search = urbancenters[urbancenters.CTR_MN_ISO == 'NLD']
search[['ID_HDC_G0', 'UC_NM_MN', 'CTR_MN_ISO', 'AREA']].sort_values('AREA', ascending=False)

,ID_HDC_G0,UC_NM_MN,CTR_MN_ISO,AREA
2122,2123.0,Rotterdam [The Hague],NLD,658.0
2166,2167.0,Amsterdam,NLD,345.0
2234,2235.0,Heerlen,NLD,129.0
2180,2181.0,Utrecht,NLD,124.0
2204,2205.0,Eindhoven,NLD,104.0
2137,2138.0,Leiden,NLD,97.0
2162,2163.0,Dordrecht,NLD,88.0
2225,2226.0,Arnhem,NLD,87.0
2182,2183.0,Tilburg,NLD,63.0
2143,2144.0,Zoetermeer,NLD,60.0
